# A vs B FlipSide Crypto Bounty



In [2]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def get_data(url):
  resp = requests.get(url)
  if resp.status_code != 200:
    raise Exception('Failed to recieve successful response')
  return pd.DataFrame(resp.json())

In [4]:
def prep_txn_data(df):
  df['date'] = pd.to_datetime(df.hour)
  df.sort_values(by='date',inplace=True)
  df['hour'] = df.date.dt.hour
  df['day'] = df.date.dt.date
  return df

In [5]:
def show_txn_bar(df, title):
  # TODO
  # - Format hover data
  fig = px.bar(df, x='date', y=['successful_transactions', 'failed_transactions'], title=title)
  fig.add_trace(go.Scatter(x=df['date'],y=df['total_transactions'],mode='markers',name='total_transactions'))
  fig.show()

In [6]:
def show_txn_heatmap(df, title):
  df_pivot = df.pivot(index='day',columns='hour',values='total_transactions').fillna(0).reset_index()
  fig = px.imshow(df_pivot,
                labels=dict(x="Hour", y="Day", color="Transactions"),
                x=df_pivot.columns,
                y=df_pivot.index,
                title=title
               )
  fig.update_xaxes(side="top")
  fig.show()

In [ ]:
bsc_counts = prep_txn_data(get_data('https://node-api.flipsidecrypto.com/api/v2/queries/50cfdf0e-3917-4193-9101-d6807ba5ba23/data/latest'))
arb_counts = prep_txn_data(get_data('https://node-api.flipsidecrypto.com/api/v2/queries/d637b241-51e2-4ab5-ab6e-c628934b49f9/data/latest'))

> **Compare transaction volume for BSC vs. Arbitrum over the past 7 days and create a simple visualization to display both. Highlight any interesting points of comparison that you see.**

To answer the prompt, it is intuitive to first look at the number of transactions per hour within the past week.

In [ ]:
show_txn_bar(bsc_counts, 'BSC Transactions per Hour')

In [ ]:
show_txn_bar(arb_counts, 'Arbitrum Transactions per Hour')

In [ ]:
show_txn_heatmap(bsc_counts, title='BSC Transactions Heat Map')

In [ ]:
show_txn_heatmap(arb_counts, title='Arbitrum Transactions Heat Map')

We can see that Arbitrum has a higher average number of transactions as compared to the BSC as shown by the graphs and heatmap above. Additionally, one can tell that Arbitrum has more consistent activity on chain as compared to BSC.

This leads us to ask a vital question:
1. Who is conducting these transactions? Is it bots? Is it retail?

To answer this question we can segment the User group into different segments and analyse them.

In [ ]:
arb_tc = get_data('https://node-api.flipsidecrypto.com/api/v2/queries/189bc8df-f4e2-4e7c-b742-0cbb67d329da/data/latest')
bsc_tc = get_data('https://node-api.flipsidecrypto.com/api/v2/queries/5b6a586b-9301-4a42-a66c-3bbd64815469/data/latest')

In [ ]:
bsc_df = bsc_tc.pivot(index='DAY',columns='G',values='C').reset_index()
fig = px.bar(bsc_df, x='DAY', y=['1','2-9','10-24','25-49','50-100','100+'], 
             title='BSC Transactions per User Group',labels={'x': 'Date', 'y':'Users per Group'})
fig.show()

In [ ]:
arb_df = arb_tc.pivot(index='DAY',columns='G',values='C').reset_index()
fig = px.bar(arb_df, x='DAY', y=['1','2-9','10-24','25-49','50-100','100+'],
             title='Arbitrum Transactions per User Group',labels={'x': 'Date', 'y':'Users per Group'})
fig.show()

In [ ]:
bsc_df['DAY'] = bsc_df['DAY'].str.slice(0, 10)
bsc_df = bsc_df.set_index('DAY')
bsc_dff = bsc_df.copy()
bsc_dff['total'] = bsc_dff.sum(axis=1)
cols = ['1', '2-9', '10-24', '25-49', '50-100','100+']
for col in cols:
  bsc_dff[col] = (bsc_dff[col] / bsc_dff['total'] * 100).round(2)
bsc_dff[cols]

G,1,2-9,10-24,25-49,50-100,100+
DAY,,,,,,
2022-06-27,83.02,15.99,0.77,0.15,0.05,0.03
2022-06-28,84.49,14.70,0.64,0.11,0.05,0.02
2022-06-29,83.55,15.45,0.76,0.15,0.06,0.03
2022-06-30,84.41,14.47,0.93,0.08,0.07,0.04
2022-07-01,83.60,15.38,0.79,0.12,0.07,0.04
2022-07-02,83.43,15.75,0.56,0.15,0.07,0.04
2022-07-03,81.18,18.01,0.59,0.13,0.06,0.03


In [ ]:
arb_df['DAY'] = arb_df['DAY'].str.slice(0, 10)
arb_df = arb_df.set_index('DAY')
arb_dff = arb_df.copy()
arb_dff['total'] = arb_dff.sum(axis=1)
cols = ['1', '2-9', '10-24', '25-49', '50-100','100+']
for col in cols:
  arb_dff[col] = (arb_dff[col] / arb_dff['total'] * 100).round(2)
arb_dff[cols]

G,1,2-9,10-24,25-49,50-100,100+
DAY,,,,,,
2022-06-27,66.36,31.30,1.89,0.23,0.07,0.16
2022-06-28,59.15,35.27,5.08,0.27,0.12,0.11
2022-06-29,52.95,40.15,6.43,0.24,0.12,0.12
2022-06-30,60.44,37.11,1.87,0.27,0.11,0.20
2022-07-01,61.37,36.09,1.88,0.29,0.12,0.26
2022-07-02,56.16,41.20,2.00,0.36,0.13,0.15
2022-07-03,54.82,41.94,2.65,0.29,0.11,0.19


We can see the break-up of the user group from the charts above. 
We see that the number of wallets performing 1 txn a day constitutes an average of 80% of BSC users, where as it constitutes 60% of arbitrum users. 

There is significant differnce in number of wallets performing 100+ txns with Arbitrum having 5x the number of wallets as compared to BSC